In [5]:
import pandas as pd
import numpy as np
import pickle
from datetime import date, timedelta
from sklearn.metrics import roc_auc_score


In [3]:
def summary_stats_table(data):
    '''
    a function to summarize all types of data
    '''
    # count of nulls
    missing_counts = pd.DataFrame(data.isnull().sum())
    missing_counts.columns = ['count_null']

    # numeric column stats
    num_stats = data.select_dtypes(include=['int64', 'float64']).describe(
    ).loc[['count', 'min', 'max', '25%', '50%', '75%']].transpose()
    num_stats['dtype'] = data.select_dtypes(include=['int64', 'float64']).dtypes.tolist()

    # non-numeric value stats
    non_num_stats = data.select_dtypes(exclude=['int64', 'float64']).describe().transpose()
    non_num_stats['dtype'] = data.select_dtypes(exclude=['int64', 'float64']).dtypes.tolist()
    non_num_stats = non_num_stats.rename(columns={"first": "min", "last": "max"})

    # merge all
    stats_merge = pd.concat([num_stats, non_num_stats], axis=0, join='outer',
                            ignore_index=False).fillna("").sort_values('dtype')

    # choose column order based on whether non_num_stats is empty
    if non_num_stats.empty:
        column_order = ['dtype', 'count', 'count_null', 'min', 'max', '25%', '50%', '75%']
    else:
        column_order = ['dtype', 'count', 'count_null', 'unique',
                        'min', 'max', '25%', '50%', '75%', 'top', 'freq']

    # only keep columns that exist (safe check)
    existing_columns = [col for col in column_order if col in stats_merge.columns or col == 'count_null']

    summary_stats = pd.merge(stats_merge, missing_counts, left_index=True,
                             right_index=True, sort=False)[existing_columns]
    return summary_stats


def reduce_mem_usage(props):
    # 计算当前内存
    start_mem_usg = props.memory_usage().sum() / 1024 ** 2
    print("Memory usage of the dataframe is :", start_mem_usg, "MB")

    # 哪些列包含空值，空值用-999填充。why：因为np.nan当做float处理
    NAlist = []
    for col in props.columns:
        # 这里只过滤了objectd格式，如果你的代码中还包含其他类型，请一并过滤
        if (props[col].dtypes != object):

            # print("**************************")
            # print("columns: ", col)
            # print("dtype before", props[col].dtype)

            # 判断是否是int类型
            isInt = False
            mmax = props[col].max()
            mmin = props[col].min()

            # Integer does not support NA, therefore Na needs to be filled
            if not np.isfinite(props[col]).all():
                NAlist.append(col)
                props[col].fillna(-999, inplace=True)  # 用-999填充

            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = np.fabs(props[col] - asint)
            result = result.sum()
            if result < 0.01:  # 绝对误差和小于0.01认为可以转换的，要根据task修改
                isInt = True

            # make interger / unsigned Integer datatypes
            if isInt:
                if mmin >= 0:  # 最小值大于0，转换成无符号整型
                    if mmax <= 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mmax <= 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mmax <= 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:  # 转换成有符号整型
                    if mmin > np.iinfo(np.int8).min and mmax < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mmin > np.iinfo(np.int16).min and mmax < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mmin > np.iinfo(np.int32).min and mmax < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mmin > np.iinfo(np.int64).min and mmax < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)
            else:  # 注意：这里对于float都转换成float16，需要根据你的情况自己更改
                props[col] = props[col].astype(np.float16)

            # print("dtype after", props[col].dtype)
            # print("********************************")
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2
    print("Memory usage is: ", mem_usg, " MB")
    print("This is ", 100*mem_usg/start_mem_usg, "% of the initial size")
    return props, NAlist


def build_dataset(raw, until, is_train):
    # Sum of daily payment for each customer
    df = raw[raw.order_pay_time > date(2013, 2, 1)]
    df['date'] = pd.DatetimeIndex(df['order_pay_time']).date
    df_payment = df[['customer_id', 'date', 'order_total_payment']]
    df_payment = df_payment.groupby(['date', 'customer_id']).agg({'order_total_payment': ['sum']})
    df_payment.columns = ['day_total_payment']
    df_payment.reset_index(inplace=True)
    df_payment = df_payment.set_index(
        ["customer_id", "date"])[["day_total_payment"]].unstack(level=-1).fillna(0)
    df_payment.columns = df_payment.columns.get_level_values(1)

    # Sum of daily order for each customer
    df_goods = df[['customer_id', 'date', 'order_total_num']]
    df_goods = df_goods.groupby(['date', 'customer_id']).agg({'order_total_num': ['sum']})
    df_goods.columns = ['day_total_num']
    df_goods.reset_index(inplace=True)
    df_goods = df_goods.set_index(
        ["customer_id", "date"])[["day_total_num"]].unstack(level=-1).fillna(0)
    df_goods.columns = df_goods.columns.get_level_values(1)

    # return the df["periods" days since "dt-minus"]
    def get_timespan(df, dt, minus, periods, freq='D'):
        return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

    X = {}
    tmp = df_payment.reset_index()
    X['customer_id'] = tmp['customer_id']
    print('Preparing payment feature...')
    for i in [14, 30, 60, 91]:
        tmp = get_timespan(df_payment, until, i, i)
        # X['diff_%s_mean' % i] = tmp_1.diff(axis=1).mean(axis=1).values
        X['mean_%s_decay' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        # X['mean_%s' % i] = tmp_1.mean(axis=1).values
        # X['median_%s' % i] = tmp.median(axis=1).values
        # X['min_%s' % i] = tmp_1.min(axis=1).values
        X['max_%s' % i] = tmp.max(axis=1).values
        # X['std_%s' % i] = tmp_1.std(axis=1).values
        X['sum_%s' % i] = tmp.sum(axis=1).values
    for i in [14, 30, 60, 91]:
        tmp = get_timespan(df_payment, until + timedelta(days=-7), i, i)
        X['mean_%s_decay_2' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        # X['mean_%s_2' % i] = tmp_2.mean(axis=1).values
        # X['median_%s_2' % i] = tmp.median(axis=1).values
        # X['min_%s_2' % i] = tmp_2.min(axis=1).values
        X['max_%s_2' % i] = tmp.max(axis=1).values
        # X['std_%s_2' % i] = tmp_2.std(axis=1).values
    for i in [14, 30, 60, 91]:
        tmp = get_timespan(df_payment, until, i, i)
        X['has_sales_days_in_last_%s' % i] = (tmp != 0).sum(axis=1).values
        X['last_has_sales_day_in_last_%s' % i] = i - ((tmp != 0) * np.arange(i)).max(axis=1).values
        X['first_has_sales_day_in_last_%s' % i] = ((tmp != 0) * np.arange(i, 0, -1)).max(axis=1).values

    # 商品数量特征，这里故意把时间和消费特征错开，提高时间滑窗的覆盖面
    print('Preparing num feature...')
    for i in [21, 49, 84]:
        tmp = get_timespan(df_goods, until, i, i)
        # X['goods_diff_%s_mean' % i] = tmp_1.diff(axis=1).mean(axis=1).values
        # X['goods_mean_%s_decay' % i] = (tmp_1 * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        X['goods_mean_%s' % i] = tmp.mean(axis=1).values
        # X['goods_median_%s' % i] = tmp.median(axis=1).values
        # X['goods_min_%s' % i] = tmp_1.min(axis=1).values
        X['goods_max_%s' % i] = tmp.max(axis=1).values
        # X['goods_std_%s' % i] = tmp_1.std(axis=1).values
        X['goods_sum_%s' % i] = tmp.sum(axis=1).values
    for i in [21, 49, 84]:
        tmp = get_timespan(df_goods, until + timedelta(weeks=-1), i, i)
        # X['goods_diff_%s_mean_2' % i] = tmp_2.diff(axis=1).mean(axis=1).values
        # X['goods_mean_%s_decay_2' % i] = (tmp_2 * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        X['goods_mean_%s_2' % i] = tmp.mean(axis=1).values
        # X['goods_median_%s_2' % i] = tmp.median(axis=1).values
        # X['goods_min_%s_2' % i] = tmp_2.min(axis=1).values
        X['goods_max_%s_2' % i] = tmp.max(axis=1).values
        X['goods_sum_%s_2' % i] = tmp.sum(axis=1).values
    for i in [21, 49, 84]:
        tmp = get_timespan(df_goods, until, i, i)
        X['goods_has_sales_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
        X['goods_last_has_sales_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values
        X['goods_first_has_sales_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values

    X = pd.DataFrame(X)

    reduce_mem_usage(X)

    if is_train:
        # 这样转换之后，打标签直接用numpy切片就可以了
        # 当然这里前提是确认付款总额没有负数的问题
        X['label'] = df_goods[pd.date_range(until, periods=30)].max(axis=1).values
        X['label'][X['label'] > 0] = 1
        return X
    return X

In [4]:
raw = pd.read_csv('train.csv')
# 处理id字段
raw['order_detail_id'] = raw['order_detail_id'].astype(np.uint32)
raw['order_id'] = raw['order_id'].astype(np.uint32)
raw['customer_id'] = raw['customer_id'].astype(np.uint32)
raw['goods_id'] = raw['goods_id'].astype(np.uint32)
raw['goods_class_id'] = raw['goods_class_id'].astype(np.uint32)
raw['member_id'] = raw['member_id'].astype(np.uint32)
# 处理状态字段，这里同时处理空值，将空值置为0
raw['order_status'] = raw['order_status'].astype(np.uint8)
raw['goods_has_discount'] = raw['goods_has_discount'].astype(np.uint8)
raw["is_member_actived"].fillna(0, inplace=True)
raw["is_member_actived"]=raw["is_member_actived"].astype(np.int8)
raw["member_status"].fillna(0, inplace=True)
raw["member_status"]=raw["member_status"].astype(np.int8)
raw["customer_gender"].fillna(0, inplace=True)
raw["customer_gender"]=raw["customer_gender"].astype(np.int8)
raw['is_customer_rate'] = raw['is_customer_rate'].astype(np.uint8)
raw['order_detail_status'] = raw['order_detail_status'].astype(np.uint8)
# 处理日期
raw['goods_list_time']=pd.to_datetime(raw['goods_list_time']).dt.date
raw['order_pay_time']=pd.to_datetime(raw['order_pay_time']).dt.date
raw['goods_delist_time']=pd.to_datetime(raw['goods_delist_time']).dt.date


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
# build the time series dataset
X_train = build_dataset(raw, date(2013, 8, 1), True)
X_train = pd.concat([X_train], axis=1)
with open('ts_train.pkl', 'wb') as f:
    pickle.dump(X_train, f)
X_test = build_dataset(raw, date(2013, 9, 1), False)
X_test = pd.concat([X_test], axis=1)
with open('ts_test.pkl', 'wb') as f:
    pickle.dump(X_test, f)

In [24]:
# load the saved datasets
with open('ts_train.pkl', 'rb') as f:
    X_train = pickle.load(f)
with open('ts_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

customer_id = X_test["customer_id"].values.tolist()
X_train.drop(['customer_id'], inplace=True, axis=1)
X_test_with_id = X_test.copy()
X_test.drop(['customer_id'], inplace=True, axis=1)

print("test shape:", X_test.shape, "train shape:", X_train.shape)
print(X_train['label'].value_counts())


test shape: (681436, 59) train shape: (681436, 60)
label
0.0    512617
1.0    168819
Name: count, dtype: int64


In [7]:
label = X_train["label"].values.tolist()
# normalize
X_train = (X_train - X_train.min()) / (X_train.max() - X_train.min())
X_test = (X_test - X_test.min()) / (X_test.max() - X_test.min())
X_train["label"] = label

In [8]:
# separate train and valid set
size = X_train.shape[0]
train_ratio = 0.8
X_valid = X_train.iloc[int(size*train_ratio):]
X_train_for_val = X_train.iloc[:int(size*train_ratio)]

customer_id = pd.DataFrame(columns=['customer_id'], data=customer_id)
customer_id.to_csv("test_customer_id.csv", index=False, header=False)

In [13]:
import lightgbm as lgb
clf = lgb.LGBMClassifier(
    num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='binary',
    max_depth=-1, learning_rate=0.005, min_child_samples=3, random_state=2021,
    n_estimators=2500, subsample=1, colsample_bytree=1,
)

In [30]:
clf.fit(X_train_for_val.drop(['label'], axis=1), X_train_for_val['label'])
y_valid_pred_proba = clf.predict_proba(X_valid.drop(['label'], axis=1))[:, 1]
auc_score = roc_auc_score(X_valid['label'], y_valid_pred_proba)
y_pre = clf.predict(X_valid.drop(['label'], axis=1))
accuracy = np.mean(y_pre == X_valid['label'])
print(f"The AUC score on the validation set is: {auc_score:.4f}")
print(f"The accuracy on the validation set is: {accuracy:.4f}")

[LightGBM] [Info] Number of positive: 132311, number of negative: 412837
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.041681 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6640
[LightGBM] [Info] Number of data points in the train set: 545148, number of used features: 59
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.242707 -> initscore=-1.137898
[LightGBM] [Info] Start training from score -1.137898
The AUC score on the validation set is: 0.6929
The accuracy on the validation set is: 0.7331


In [17]:
clf.fit(X_train.drop(['label'], axis=1), X_train['label'])


[LightGBM] [Info] Number of positive: 168819, number of negative: 512617
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049159 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6971
[LightGBM] [Info] Number of data points in the train set: 681436, number of used features: 59
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.247740 -> initscore=-1.110702
[LightGBM] [Info] Start training from score -1.110702


LGBMClassifier(colsample_bytree=1, learning_rate=0.005, min_child_samples=3,
               n_estimators=2500, objective='binary', random_state=2021,
               reg_alpha=0.25, reg_lambda=0.25, subsample=1)

In [32]:
y_pred = clf.predict_proba(X_test)[:, 1]
df_preds = pd.DataFrame(
    {
        "customer_id": X_test_with_id['customer_id'].values,
        "pred": y_pred.flatten()
    }
)
# 读入待提交文件
sub = pd.read_csv('submission.csv')

# 合并预测结果
submission = pd.merge(sub, df_preds, on='customer_id', how='left')
submission.fillna(0,inplace=True)
submission = submission[['customer_id','pred']]
submission.rename(columns={'customer_id':'customer_id','pred':'result'}, inplace=True)

# 将概率值转换为用户是否购买的标签
def f(x):
    if x <= 0.4:   # 调整阈值
        return 0
    else:
        return 1

submission['result'] = submission['result'].map(f)
value_count = submission['result'].value_counts()
print(value_count)

# 保存结果
submission.to_csv('submission_test_ts.csv',index=False)

result
0    1469416
1     116570
Name: count, dtype: int64
